In [1]:
using Pkg
Pkg.add("StatsBase")
Pkg.add("ClassImbalance")
Pkg.add("ROCAnalysis")
using DataFrames
using CSV
using Plots
using GLM
using StatsBase
using Lathe
using MLBase
using ClassImbalance
using ROCAnalysis

#enable printing of 1000 columns
ENV["COLUMNS"] = 1000


    Updating registry at `C:\Users\Lavish Arora\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
    Updating `C:\Users\Lavish Arora\.julia\environments\v1.6\Project.toml`
  [2913bbd2] + StatsBase v0.33.6
  No Changes to `C:\Users\Lavish Arora\.julia\environments\v1.6\Manifest.toml`
Precompiling project...
  ✓ IntelOpenMP_jll
  ✓ EarCut_jll
  ✓ LZO_jll
  ✓ LAME_jll
  ✓ OpenSpecFun_jll
  ✓ libpng_jll
  ✓ Libgpg_error_jll
  ✓ Rmath_jll
  ✓ OpenSSL_jll
  ✓ FriBidi_jll
  ✓ Libmount_jll
  ✓ libfdk_aac_jll
  ✓ Xorg_libXdmcp_jll
  ✓ x264_jll
  ✓ JpegTurbo_jll
  ✓ PCRE_jll
  ✓ Opus_jll
  ✓ Libiconv_jll
  ✓ Bzip2_jll
  ✓ Expat_jll
  ✓ Xorg_libXau_jll
  ✓ LibVPX_jll
  ✓ Zstd_jll
  ✓ Ogg_jll
  ✓ Xorg_xtrans_jll
  ✓ Libffi_jll
  ✓ x265_jll
  ✓ Libuuid_jll
  ✓ Pixman_jll
  ✓ FFTW_jll
  ✓ Xorg_libpthread_stubs_jll
  ✓ MKL_jll
  ✓ Libgcrypt_jll
  ✓ Rmath
  ✓ XML2_jll
  ✓ FreeType2_jll
  ✓ Libtiff_jll
  ✓ libvorbis_jll
 

  [efe28fd5] ↑ OpenSpecFun_jll v0.5.3+4 ⇒ v0.5.4+0
Precompiling project...
  ✓ UnPack
  ✓ PositiveFactorizations
  ✓ Calculus
  ✓ FillArrays
  ✓ PooledArrays
  ✓ ArrayInterface
  ✓ CommonSubexpressions
  ✓ DataStructures
  ✓ OffsetArrays
  ✓ Parameters
  ✓ OpenSpecFun_jll
  ✓ PDMats
  ✓ SortingAlgorithms
  ✓ StaticArrays
  ✓ QuadGK
  ✓ NearestNeighbors
  ✓ StatsBase
  ✓ SpecialFunctions
  ✓ CategoricalArrays
  ✓ DiffResults
  ✓ Contour
  ✓ DiffEqDiffTools
  ✓ Interpolations
  ✓ CSV
  ✓ Clustering
  ✓ MultivariateStats
  ✓ MLBase
  ✓ DiffRules
  ✓ StatsFuns
  ✓ ColorSchemes
  ✓ StatsModels
  ✓ GeometryBasics
  ✓ ForwardDiff
  ✓ DataFrames
  ✓ NLSolversBase
  ✓ Distributions
  ✓ Lathe
  ✓ GLM
  ✓ LineSearches
  ✓ PlotUtils
  ✓ ClassImbalance
  ✓ Optim
  ✓ RecipesPipeline
  ✓ KernelDensity
  ✓ PlotThemes
  ✓ Plots
  ✓ StatsPlots
47 dependencies successfully precompiled in 124 seconds (133 already precompiled)
   Resolving package versions...
   Installed ROCAnalysis ─ v0.3.2
    Updating 

1000

# LOADING THE DATA

In [2]:
df = DataFrame(CSV.File("Churn_Modelling.csv"))
first(df,10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
,Int64,Int64,String,Int64,String,String,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64
1,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,1.01349e5,1
2,2,15647311,Hill,608,Spain,Female,41,1,83807.9,1,0,1,1.12543e5,0
3,3,15619304,Onio,502,France,Female,42,8,1.59661e5,3,1,0,1.13932e5,1
4,4,15701354,Boni,699,France,Female,39,1,0.0,2,0,0,93826.6,0
5,5,15737888,Mitchell,850,Spain,Female,43,2,1.25511e5,1,1,1,79084.1,0
6,6,15574012,Chu,645,Spain,Male,44,8,1.13756e5,2,1,0,1.49757e5,1
7,7,15592531,Bartlett,822,France,Male,50,7,0.0,2,1,1,10062.8,0
8,8,15656148,Obinna,376,Germany,Female,29,4,1.15047e5,4,1,0,1.19347e5,1
9,9,15792365,He,501,France,Male,44,4,1.42051e5,2,0,1,74940.5,0


In [3]:
println(size(df))
describe(df)

(10000, 14)


,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,RowNumber,5000.5,1,5000.5,10000,,,Int64
2,CustomerId,1.56909e7,15565701,1.56907e7,15815690,,,Int64
3,Surname,,Abazu,,Zuyeva,2932,,String
4,CreditScore,650.529,350,652.0,850,,,Int64
5,Geography,,France,,Spain,3,,String
6,Gender,,Female,,Male,2,,String
7,Age,38.9218,18,37.0,92,,,Int64
8,Tenure,5.0128,0,5.0,10,,,Int64
9,Balance,76485.9,0.0,97198.5,2.50898e5,,,Float64


In [5]:
#check names of columns

names(df)

14-element Vector{Symbol}:
 :RowNumber
 :CustomerId
 :Surname
 :CreditScore
 :Geography
 :Gender
 :Age
 :Tenure
 :Balance
 :NumOfProducts
 :HasCrCard
 :IsActiveMember
 :EstimatedSalary
 :Exited

In [7]:
#count the classes

countmap(df.Exited)

#countmap function counts the number of unique entries in the dataframe's column

Dict{Int64, Int64} with 2 entries:
  0 => 7963
  1 => 2037

# PREPROCESSING

In [10]:
Lathe.preprocess.OneHotEncode(df,:Geography)
Lathe.preprocess.OneHotEncode(df,:Gender)
select!(df,Not([:RowNumber,:CustomerId,:Surname,:Geography,:Gender,:Male]))
first(df,5)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Spain,Germany,Female
,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64,Bool,Bool,Bool,Bool
1,619,42,2,0.0,1,1,1,1.01349e5,1,1,0,0,1
2,608,41,1,83807.9,1,0,1,1.12543e5,0,0,1,0,1
3,502,42,8,1.59661e5,3,1,0,1.13932e5,1,1,0,0,1
4,699,39,1,0.0,2,0,0,93826.6,0,1,0,0,1
5,850,43,2,1.25511e5,1,1,1,79084.1,0,0,1,0,1


# TRAIN TEST SPLITTING

In [12]:
using Lathe.preprocess: TrainTestSplit
train,test = TrainTestSplit(df,.75)

(7471×13 typename(DataFrame)
│ Row  │ CreditScore │ Age   │ Tenure │ Balance   │ NumOfProducts │ HasCrCard │ IsActiveMember │ EstimatedSalary │ Exited │ France │ Spain │ Germany │ Female │
│      │ Int64       │ Int64 │ Int64  │ Float64   │ Int64         │ Int64     │ Int64          │ Float64         │ Int64  │ Bool   │ Bool  │ Bool    │ Bool   │
├──────┼─────────────┼───────┼────────┼───────────┼───────────────┼───────────┼────────────────┼─────────────────┼────────┼────────┼───────┼─────────┼────────┤
│ 1    │ 608         │ 41    │ 1      │ 83807.9   │ 1             │ 0         │ 1              │ 1.12543e5       │ 0      │ 0      │ 1     │ 0       │ 1      │
│ 2    │ 502         │ 42    │ 8      │ 1.59661e5 │ 3             │ 1         │ 0              │ 1.13932e5       │ 1      │ 1      │ 0     │ 0       │ 1      │
│ 3    │ 850         │ 43    │ 2      │ 1.25511e5 │ 1             │ 1         │ 1              │ 79084.1         │ 0      │ 0      │ 1     │ 0       │ 1      │
│ 4    │ 82

# MODEL BUILDING

In [15]:
fm = @formula(Exited ~ CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember +EstimatedSalary + Female + France + Spain)
logit = glm(fm,train,Binomial(),ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.Cholesky{Float64, Matrix{Float64}}}}, Matrix{Float64}}

Exited ~ 1 + CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France + Spain

Coefficients:
────────────────────────────────────────────────────────────────────────────────────────
                        Coef.   Std. Error       z  Pr(>|z|)     Lower 95%     Upper 95%
────────────────────────────────────────────────────────────────────────────────────────
(Intercept)      -1.79027      0.166129     -10.78    <1e-26  -2.11588      -1.46467
CreditScore      -0.000502859  0.000183259   -2.74    0.0061  -0.000862041  -0.000143677
Age               0.0417637    0.00168492    24.79    <1e-99   0.0384613     0.0450661
Tenure           -0.00501467   0.00611284    -0.82    0.4120  -0.0169956     0.00696628
Balance         

In [17]:
#predict the target variable on test data

prediction = predict(logit,test)

2529-element Vector{Union{Missing, Float64}}:
 0.12880494697362008
 0.22090102526662703
 0.2658345189453921
 0.2721298669423189
 0.1415765046240161
 0.13142710267869723
 0.04145360270867354
 0.04315615198545685
 0.20921594822203213
 0.5310406118532094
 0.3875971104533673
 0.30908266506931426
 0.09633818680484246
 ⋮
 0.05406128260304856
 0.4279482278445095
 0.05053435190352962
 0.1898757874125937
 0.221806407473504
 0.22110024599332503
 0.08703064888604659
 0.19962227668141092
 0.15153881244280912
 0.10812123758124453
 0.5004146051850895
 0.097813562283321